# Exploratory Analysis of Accident Data Using Plotly Visualization Tool

# Outline
1. Introduction
2. Visualization Technique
3. Visualization Library
    1. Data Cleaning
    2. Visualizations
4. Summary
5. Rules of Rule et al

# Introduction

<p>
    The purpose of the analysis is to provide and example to work off of demonstrating the plotly visualization library. We will be using accident data provided for 2018 provided by the National Highway Transportation and Safety Administration (NHTSA).There will be a few cells dedicated to showing how the data was cleaned then followed examples of visulizations of that data with plotly. The question we want to answer is as follows.
    </p>
    
<i> How does lead time from when first responders are notified about an accident to when accident victims arrive at the hospital, compare between rural areas and urban areas?  </i>

#### The Dataset
<p>
    The NHTSA provides a yearly conglomeration of accident data collected by state agencies. The provide it as SAS and .csv formats. The data comes in various files that contain different variables. The have a primary key typically that allows you to join certain incidents with variables from other files. The dataset is further described in the coding manual provided in the projects top directory. 
</p>

#### Libraries and Dependencies
To reproduce this notebook you will need several libraries and some code to setup your environment. It is currently running a **python 3.7** interpreter inside a virtual environment. A github repo with readme.md has been provided to properly setup your machine to execute notebook. Assuming you have gotten this far where you are reading this markdown then you are ready to proceed. Lets quickly take a look at some of the information about the requirements and dependencies. 

In [89]:
# from the requirments.txt for the virtual environment 
# many of the others come with installation from the 
! cat requirements.txt | grep -E -- "plotly|numpy|pandas" 
! python3 --version

numpy==1.17.4
pandas==0.25.3
plotly==4.3.0
Python 3.7.3


In [138]:
# lets import those into our file
import pandas as pd
import numpy as np
import plotly

# this gives us a clean way to visualize tables inline with the notebook
from IPython.display import display

# Visualization Technique

### The Violin Plot
The visualization were going to use to answer the question were after is called a **violin plot**. It is often used to show the distribution of a certain quantitative variable over multiple categories. Which is exactly what we intend to do with this question. Below is a diagram showing the anatomy of a violin plot. 

<img src="violin_plot.svg" width="400">
<center><i>Photo Credit: https://datavizcatalogue.com/methods/violin_plot.html </i></center>

You can see that it conveys a lot of the same information as a box plot, and you often see the two coupled together in the same plot. The additional measurement that you get with the violin plot compared to just the box plot is the probability estimation (using kernal density estimation for smoothing out curves).

Instead of a violin plot one could use a box plot and convey essentially the same information. Coupled with some sort of histogram or probability estimation like a density plot. This plot should not be used when trying to compare magnitudes of data or counts of certain variables. A bar chart may work better in those circumstances. 



# Visualization Library

#### Plotly
The plotly visualization library is an open source framework provided under the MIT license. It is free to use and shares it repo on github [here](https://github.com/plotly/plotly.py). The project is active current release version is 4.3.0 and came out early November 2019 and has had 24 pull requests merged in the last month from this posting. This library has a really well documented api which makes it very easy to get up and running quickly with stunning visualizations. Visit the documentation [here](https://plot.ly/python/). It's statistical library is very robust as well offering violin plots, histograms, box plots, and [more](https://plot.ly/python/#statistical-charts). The plotly library describes charts and plots declaritvely. Where all aspects of a chart or figure are set using key value pairs. A reference to all those attributes can be found [here](https://plot.ly/python/reference/). 

I chose to use the library because of how fully baked it is and how simple the syntax is. It is intuitive in its approach and compares well to matplotlib and altiar libraries. I have used plotly before for other projects and really enjoyed the experience and built in interactivity with overlays. It's default color schemes are also very appealing compared to matplotlib and altair. Which give it a leg up again for ease of use. 

Plotly also has great support with jupyter notebook as well. It can make use of [ipython widgets](https://plot.ly/python/figurewidget-app/) as well as has built in [renderers](https://plot.ly/python/renderers/) for jupyter notebook  

Plotly was created by Alex Johnson, Jack Parmer, Chris Parmer, and Matthew Sundquist. And more information about plotly and the company can be found at their [WikiPedia Page](https://en.wikipedia.org/wiki/Plotly).

# Demonstration 

## 1. Data Cleaning

In this section I am demonstrating cleaning of the raw data from the .csv files to an in memory pandas dataframe. In the process I perform some various techniques to make the data look the way we need it to. Those techniques and tasks are highlighted below. 

In [170]:
# lets import the maps.py data one such variable is make this may be complicated approach and maybe a bad one
from maps import *
import importlib
# importlib.reload(maps)

#### 1(a) Import Data
Here we are importing the data from the CSVs into memory and only selecting the columns we would like. Then we join all of those into a single dataframe for analysis.

In [185]:
# read in the data from Accident/Vehicle csv files
# here we specify the variables to read from the csv for each file
acc_cols = ['ST_CASE', 'STATE', 'MONTH','YEAR', 'FATALS', 'DAY', 'DAY_WEEK', 'HOUR', 'MINUTE', 'NOT_HOUR', 'NOT_MIN', 'ARR_HOUR', 'ARR_MIN', 'HOSP_HR', 'HOSP_MN', 'ROUTE', 'RUR_URB']
veh_cols = ['ST_CASE', 'MAKE','DR_HGT', 'DR_WGT']

acc_df = pd.read_csv("FARS2018NationalCSV/ACCIDENT.csv", usecols = acc_cols)
veh_df = pd.read_csv("FARS2018NationalCSV/VEHICLE.csv", usecols = veh_cols)

df = pd.merge(acc_df, veh_df, on='ST_CASE', how = 'inner')
display(df.head())

,STATE,ST_CASE,DAY,MONTH,YEAR,DAY_WEEK,HOUR,MINUTE,RUR_URB,ROUTE,NOT_HOUR,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,FATALS,MAKE,DR_HGT,DR_WGT
0,1,10001,5,1,2018,6,6,0,1,1,6,99,6,15,88,88,1,82,69,210
1,1,10002,8,1,2018,2,0,48,2,1,0,99,0,59,88,88,2,58,68,140
2,1,10003,8,1,2018,2,22,50,1,1,99,99,23,10,99,99,1,63,68,185
3,1,10003,8,1,2018,2,22,50,1,1,99,99,23,10,99,99,1,7,74,200
4,1,10004,9,1,2018,3,13,2,1,1,13,99,13,14,88,88,1,49,74,250


#### 1(b) Rename Values

Here we are simply renaming values in the columns that have maps for them. Working with government datasets often require either building maps like this manually or using some strong regex and inference to provide human readable meaning from columns or values. Typically the values are stored as numbers to save on drive storage requirements. You also do not need to rename the columns to perform the analysis just makes it nice to be able to see what the numbers mean. 

In [186]:
# now lets map the columns so that they contain the values from the map so we can do our analysis
df.MAKE = df.MAKE.map(make_map())
# df.MONTH = df.MONTH.map(month_map())
df.DAY_WEEK = df.DAY_WEEK.map(day_map())
df.ROUTE = df.ROUTE.map(route_map())
df.RUR_URB = df.RUR_URB.map(rur_urb_map())
df.STATE = df.STATE.map(state_map())
# display(df.head())

#### 1(c) Remove Rows That Have Unknown Time Frames
The dataset has some rows that have unknown times for various events in the timeline. We are going to remove those for the primary analysis but store the original dataframe just in case we want to perform further analysis.


In [187]:
# Let's build some code to make this happen
# These variables will come in handy for the next few items
acc_time_cols = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']
time_event_cols = {
    'ACC_TIME': acc_time_cols,
    'NOT_TIME':acc_time_cols[:3]+["NOT_HOUR","NOT_MIN"], 
    'ARR_TIME':acc_time_cols[:3]+["ARR_HOUR", "ARR_MIN"], 
    'HOSP_TIME':acc_time_cols[:3]+["HOSP_HR", "HOSP_MN"]
    }

# lets modify the dataframe by dropping the rows that have unknown values
len1 = len(df)
for k,v in time_event_cols.items():
    if k != 'ACC_TIME':
        for c in v[4:]:
            df = df[df[c]<88]
len2 = len(df)
print(f'''started with: {len1}
ended with {len2}
rows dropped = {len1-len2}'''
)

started with: 51872
ended with 13821
rows dropped = 38051


#### 1(d) Rectify Time Columns to Date Time Object. 

So each event that we are going to track a timestamp of has a column dedicated to describe the time unit. We are going to rectify that by simply taking each column of interest into a dataframe then renaming the columns to something like min, hour, year, etc. We will take that newly created dataframe and send it to datetime object using pandas built in libraries. The result it a series that were going to add on to the existing dataframe. Then we will drop the columns we do not need for analysis.

In [204]:
# this function will take the columns and rename them then
# then return the time series data
# make time series is also not making a time series but making a pd.Series datetime object hehe
def make_time_series(df, out_name):
    edges = {'MN':'minute','MIN': 'minute', 'MO': 'month', 'DA': 'day', 'YR': 'year', 'HR':'hour'}
    df.columns = [n.split('_')[1] if '_' in n else n for n in df.columns]
    df = df.rename(columns=edges)
    s = pd.to_datetime(df)
    s.name = out_name
    return s

for k,v in time_event_cols.items():
    if k != 'ACC_TIME':
        df[k] = make_time_series(df[v], k)
    else:
        df[k] = pd.to_datetime(df[v])

#lets drop unnecessary columns now
cols_ = ['ST_CASE', 'STATE', 'FATALS','RUR_URB', 'ROUTE', 'MAKE', 'ACC_TIME', 'NOT_TIME', 'ARR_TIME', 'HOSP_TIME', 'DR_HGT', 'DR_WGT']
df_final = df.drop(columns = df.columns.difference(cols_))
display(df_final.head(5))



,STATE,ST_CASE,RUR_URB,ROUTE,FATALS,MAKE,DR_HGT,DR_WGT,ACC_TIME,NOT_TIME,ARR_TIME,HOSP_TIME
64,Alabama,10044,Rural,State Highway,2,Jeep/Kaiser-Jeep/Willys Jeep,64,140,2018-01-15 15:35:00,2018-01-15 15:52:00,2018-01-15 16:36:00,2018-01-15 17:40:00
65,Alabama,10044,Rural,State Highway,2,Toyota,73,210,2018-01-15 15:35:00,2018-01-15 15:52:00,2018-01-15 16:36:00,2018-01-15 17:40:00
74,Alabama,10049,Rural,U.S.Highway,1,Pontiac,58,138,2018-01-17 13:45:00,2018-01-17 13:49:00,2018-01-17 14:02:00,2018-01-17 15:46:00
77,Alabama,10051,Rural,County Road,1,Ford,75,240,2018-01-18 15:30:00,2018-01-18 15:24:00,2018-01-18 16:14:00,2018-01-18 17:14:00
85,Alabama,10058,Urban,County Road,1,Mitsubishi,71,188,2018-01-21 06:12:00,2018-01-21 06:13:00,2018-01-21 06:50:00,2018-01-21 08:04:00


## 2. Visualization

Now that we have our DataFrame nice and tidy for analysis we can start reformatting it to gain insight about the lead times between certain critical response events relative to the accident. We are going to create a few series and data frames to answer the below subsequent questions. 


In [205]:
# here we import the plotly graph objects 
# since were creating a graphic object Figure containing Violin plot. 
import plotly.graph_objects as go

#### Simple violin plot for starters

Were going to look at the distribution of Driver Height in our dataset per accident. This is not particularly interesting but gives a great introduction to using the violin plot with plotly. 

In [316]:
# here we import the plotly graph objects 
# since were creating a graphic object Figure containing Violin plot. 
import plotly.graph_objects as go

fig_0 = go.Figure()

fig_0.add_trace(
    go.Violin(
        y = df_final[df_final['DR_HGT'] <= 107]['DR_HGT']
    )
)

#### Driver Height and Weight Distribution Violin Plot

Let's try a plot with a little more complexity. So were going to plot the Height and Weight of drivers who had accidents. But let's only compare those for drivers whose accidents occured in Rural or Urban areas using multiple plots.

In [317]:
# let's drop some unknown heights and weights
# values came from the coding guide...
df_msr = df_final[(df_final['DR_HGT']<=107) & (df_final['DR_WGT']<=700)]


# create the figure object
fig_1 = go.Figure()

for msr in ['DR_HGT', 'DR_WGT']:
    title = ""
    fig_title = ""
    if msr == 'DR_HGT':
        title = 'Height inches'
        fig_title = "<b> Height of drivers in Rural vs Urban Areas"
    else:
        title = 'Weight lbs'
        fig_title = "<b> Weight of drivers in Rural vs Urban Areas"
        
    fig_1.add_trace(
        go.Violin(
            x=df_msr[df_msr['RUR_URB']=='Rural']['RUR_URB'],
            y=df_msr[df_msr['RUR_URB']=='Rural'][msr], 
            name = 'Rural',
            legendgroup='Yes', 
            scalegroup='Yes',
            line_color='blue'
        )
    )
    fig_1.add_trace(
        go.Violin(
            x=df_msr[df_msr['RUR_URB']=='Urban']['RUR_URB'],
            y=df_msr[df_msr['RUR_URB']=='Urban'][msr], 
            name = 'Urban',
            legendgroup='Yes', 
            scalegroup='Yes',
            line_color='red'
        )
    )
    fig_1.update_traces(meanline_visible=True, box_visible=True)
    fig_1.update_layout(
        violingap=0, 
        violinmode='overlay', 
        yaxis_title=title,
        title = fig_title
    )
    fig_1.show()


Above we can see that the weights and heights are about the same for drivers who had accidents in Rural vs Urban areas and their distributions are roughly the same. This is could be the expected result given weights and heights may not vary much between rural and urban areas, and there is no reason that a person from a rural area could not get in an accident in an urban environment. 

The interesting thing to note is how you can place the fig.add_trace() object inside a for loop and display multiple traces on one plot as well as show two identical plots with different data. 

#### Rural vs. Urban Lead Time to Hospital

We want to see if there is difference in lead time between when an accident is reported to when the patient arrives at the hospital. This is particularly interesting analysis to see if rural agencies or urban agencies perform better at responding to accidents.

In [256]:
# here we initialize the figure object. 
fig_2 = go.Figure()

rur_urb = df['RUR_URB'].unique()
for ru in rur_urb:
    x = df_final[df_final['RUR_URB']==ru]['RUR_URB']
    y = (df_final[df_final['RUR_URB']==ru]['HOSP_TIME']-df_final[df_final['RUR_URB']==ru]['NOT_TIME']).astype('timedelta64[m]')
    y = y[y.between(y.quantile(0.10), y.quantile(0.90))]
    
    fig_2.add_trace(
        go.Violin(
            x=x,
            y=y,
            name=ru,
            box_visible=True,
            meanline_visible=True,
        )
    )
fig_2.update_layout(
    title_text="<b>Time Between Accident Notification<br>And Arrival At Hospital</b><br><i>",
    violingap=0.2, 
    violingroupgap=0.1, 
    violinmode='overlay',
    yaxis_title="Minutes Notification - Hospital",
)
fig_2.show()

The plots present interesting results. It appears that Rural first responers may have longer lead times between getting a notification and having a victim arrive at the hospital. 

Again notice how using loops can add traces to the same plot and present variables side by side on the same plot space.

#### Rural vs Urban Lead Times by State (Top N)
Were going to now view the Rural vs Urban lead times from notification to the hospital for the top N states. This time were going to add all the traces to the same plot and they will contain rural vs lead times as a "split violin" plot. Giving us comparisons between states and rural vs urban areas. 

So our X axis will contain the N number of states. Y will be the distribution Left violin half will be the Rural distribution of lead time and the Right violin half will be the urban distribution of lead time. 

In [318]:
# lets first determine top N states with most accidents
states = df_final.groupby('STATE')['STATE'].count().sort_values(ascending = False)
n = 6 
n_states = states.index[:n]

In [319]:
# create the fig
fig_3 = go.Figure()

# build function to create and return the violin object
def violin(state, rur_urb, leg):
    # some context based variables
    if rur_urb == 'Rural':
        lc = 'red'
        side = 'negative'
    if rur_urb == 'Urban':
        lc = 'blue'
        side = 'positive'
    # masks for getting data from df_final and calculating the y could be cleaner 
    x = df_final[(df_final['STATE'] == state) & (df_final['RUR_URB']== rur_urb)]['STATE']
    y = (df_final[(df_final['STATE'] == state) & (df_final['RUR_URB']==rur_urb)]['HOSP_TIME']-df_final[(df_final['STATE'] == state) & (df_final['RUR_URB']== rur_urb)]['NOT_TIME']).astype('timedelta64[m]')
    # get rid of some edge cases
    y = y[y.between(y.quantile(0.10), y.quantile(0.90))]
    #return the trace object
    return go.Violin(
        x = x,
        y = y,
        name=rur_urb,
        line_color = lc,
        legendgroup = rur_urb,
        side = side,
        box_visible=True,
        meanline_visible=True,
        showlegend = leg
    )

# generate the traces to add to the fig
for i, s in enumerate(n_states):
    leg = True if i < 1 else False
    for ru in ['Rural', 'Urban']:
        fig_3.add_trace(violin(s, ru, leg))

# lets add some additional figure style
fig_3.update_layout(
    title_text="<b>Lead Time Rural vs Urban By State",
    violingap=0.2, 
    violingroupgap=0, 
    violinmode='overlay',
    yaxis_title="Lead Time Minutes",
    xaxis_title = 'States',
)

fig_3.show()
    


Looks like the response times for rural vs urban remain consistent in terms of there being in general faster lead times in urban areas vs rural areas, as well as urban areas having a more central (less varied) distribution. As the chart indicates it looks like some states vary in their lead times. Although texas has more accidents it appears the median lead times are not as fast in urban areas compared to some of the other states. 

Using plotly to generate several violin plot traces is as simple as creating the traces objects in an iterative fashion. You can get some very nice data analysis using split violin plots and creating several trace objects. 

# Summary

The datasets provided by the NHSTA provide a great playground for exploring data analysis tools and visualizations. If you are intrigued about finding information about the safety of your road systems they provide data down to the county level. This is just a small and succint example of what sort of information you can glean from their datasets. 

Plotly, pandas, and python can enable you to make some powerful and stunning visualizations. The declarative nature makes it easy to generate multiple plots and figures. I highly recommend this library as it is intuitive and powerful. 

# Ten Rules of Rule. et al

Rule 1: Tell a story for an audience
Rule 2: Document the process, not just the results
Rule 3: Use cell divisions to make steps clear
Rule 4: Modularize code
Rule 5: Record dependencies
Rule 6: Use version control
Rule 7: Build a pipeline
Rule 8: Share and explain your data
Rule 9: Design your notebooks to be read, run, and explored
Rule 10: Advocate for open research

Of the ten rules highlighted above. I believe I adhered to the rules 1, 2, 3, 4 (when appropriate), 5, 6, 8. Rules 5 and 6 are evident in the usage of virtual environment, repo, and installation documentation provided. Rule 1 I tried to use an approach describing the data analysis and what steps we were taking to prune data, as well as provide a entry level to more advanced approach for plotting the violin plot. Rule 3 was evident with breaking up of code cells by functionality and their intended purpose with markdown cells in between to explain and summarize the code cell gaps. Rule 8 was followed loosely I provided the data set in the repo but did not explain the whole data however I provided links to the coding guides. The data is very complex to explain. 